# Imports

In [1]:
import pandas as pd
import random

# Files

In [2]:
df = pd.read_pickle("finaljuegos_1.pkl", compression="gzip")
df = df.iloc[:, 1:]

In [3]:
print(df.shape)
print(df.drop_duplicates("name").shape)

(60247, 63)
(60227, 63)


In [4]:
df.drop_duplicates("name", inplace = True)

# Preprocesamiento



In [5]:
df_1=df[["name","background_image", "rating", "released", "tag_names", "tag_names_str"]]

In [6]:
df.drop(["background_image", "rating", "released", "tag_names", "tag_names_str"], axis = 1, inplace = True)

In [7]:
usuario_input = [{"name" : df.sample()["name"].values[0], "rating" : random.randint(1, 5)} for x in range(20)]

In [8]:
usuario = pd.DataFrame.from_dict(usuario_input)

In [9]:
df[df["name"].isin(usuario["name"].to_list())]

,name,playtime,Android,Apple II,Game Boy,Game Boy Advance,Game Boy Color,Linux,Nintendo 3DS,Nintendo 64,...,Epic Games,GOG,Google Play,Nintendo Store,PlayStation Store,Steam,Xbox 360 Store,Xbox Store,Multiplayer,Singleplayer
1076,Supreme Commander 2,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0
4539,forma.8,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0
6156,Zombotron,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
14414,Streamer Life Simulator,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
15116,The Survey,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
19606,I-Fluid,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
21074,Ballistic Tanks,3,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
23093,Friday Night Bullet Arena,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
25175,NieR Re[in]carnation,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
25290,Ikari Warriors (1986),0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0


In [10]:
weighted_genre_matrix = (df[df["name"].isin(usuario["name"])].iloc[:, 1:].values.T * usuario["rating"].values).T

In [11]:
weighted_genre_matrix = weighted_genre_matrix.sum(axis = 0)

In [12]:
user_input = pd.DataFrame.from_dict({col : [val] for col, val in zip(df.columns[1:], weighted_genre_matrix)})

In [13]:
user_input = user_input/user_input.sum(axis = 1)[0]

In [14]:
consola_usuario = "Android"

In [15]:
juegos_matriz = df[(~df["name"].isin(usuario["name"])) & (df[consola_usuario] == 1)]

In [16]:
df_recom = pd.DataFrame()

df_recom["title"] = juegos_matriz["name"]

df_recom["valor"] = (juegos_matriz.iloc[:, 1:].values*user_input.values).sum(axis = 1)

In [17]:
df_recom.sort_values("valor", ascending = False).head(10)

,title,valor
4322,Pokémon GO,52.434783
6734,osu!,40.219368
19346,The Simpsons: Tapped Out,25.952569
2381,ROBLOX,25.221344
45381,Caller's Bane,7.660079
3242,Cookie Clicker,7.501976
2921,NBA 2K18,7.274704
23574,Strike Team Hydra,7.183794
55636,Broadway: 1849,7.136364
34830,Fallen London,7.104743


# Recomendador

In [20]:
def recomendar_juegos(df):
    consolas_usuario = []
    while True:
        consola = input("Escribe el nombre de una consola que poseas o escribe 'Listo' si terminaste: ")
        if consola.lower() == "listo":
            break
        if consola not in df.columns:
            print(f"{consola} no es una consola válida en el DataFrame.")
            continue
        consolas_usuario.append(consola)
    if not consolas_usuario:
        return "Debes introducir al menos una consola para obtener recomendaciones."
    juegos_rating_usuario = []
    while True:
        juego = input("Escribe el nombre de un juego que hayas jugado o escribe 'Listo' si terminaste: ")
        if juego.lower() == "listo":
            break
        rating = float(input(f"¿Qué valor le das a '{juego}' del 1 al 5?: "))
        try:
            rating = float(rating)
            if rating < 1 or rating > 5:
                print("El valor debe ser un número entre 1 y 5.")
                continue
        except ValueError:
            print("El valor debe ser un número.")
            continue
        juegos_rating_usuario.append((juego, rating))
     
    
    no_consolas = ['name', 'playtime','Action','Adventure','Arcade','Board Games','Card','Casual','Educational',
                   'Family','Fighting','Indie','Massively Multiplayer','Platformer','Puzzle','RPG','Racing','Shooter',
                   'Simulation','Sports','Strategy','App Store','Epic Games','GOG','Google Play','Nintendo Store',
                   'PlayStation Store','Steam','Xbox 360 Store','Xbox Store','Multiplayer','Singleplayer']
    
    columnas_final = no_consolas + consolas_usuario
    
    juegos_usuario = pd.DataFrame(juegos_rating_usuario, columns=["name", "rating"])
    
    juegos_matriz = df[~df["name"].isin(juegos_usuario["name"])][columnas_final]
    
    for col in consolas_usuario:
        juegos_matriz = juegos_matriz[juegos_matriz[col] == 1]
    
    
    if juegos_matriz.empty:
        return "No hay juegos recomendados para estas consolas."
       
    df_usuario = df[df["name"].isin(juegos_usuario["name"].to_list())]
    
    weighted_genre_matrix = (df_usuario.iloc[:, 1:].values.T * juegos_usuario["rating"].values).T
    
    weighted_genre_matrix = weighted_genre_matrix.sum(axis=0)
    
    user_input = pd.DataFrame.from_dict({col: [val] for col, val in zip(df.columns[1:], weighted_genre_matrix)})[columnas_final[1:]]
    
    user_input = user_input / user_input.sum(axis=1)[0]
    
    df_recom = pd.DataFrame()
    
    df_recom["name"] = juegos_matriz["name"]
    df_recom["valor"] = (juegos_matriz.iloc[:, 1:].values * user_input.values).sum(axis=1)
    
    df_recom = df_recom.head(40)
    
    df_recom = pd.merge(left = df_recom, right = df_1, on = "name", how = "inner").drop_duplicates()
   
    return df_recom.sort_values("valor", ascending = False).reset_index(drop = True)

In [21]:
recomendar_juegos(df)

Escribe el nombre de una consola que poseas o escribe 'Listo' si terminaste: Android
Escribe el nombre de una consola que poseas o escribe 'Listo' si terminaste: Listo
Escribe el nombre de un juego que hayas jugado o escribe 'Listo' si terminaste: Dota 2
¿Qué valor le das a 'Dota 2' del 1 al 5?: 5
Escribe el nombre de un juego que hayas jugado o escribe 'Listo' si terminaste: Listo


,name,valor,background_image,rating,released,tag_names,tag_names_str
0,Minecraft,20.388889,https://media.rawg.io/media/games/b4e/b4e4c73d...,4.42,2009-05-10,"['Multiplayer', 'Sandbox', 'Crafting', 'Buildi...","Multiplayer, Sandbox, Crafting, Building, 3D, ..."
1,Terraria,11.055556,https://media.rawg.io/media/games/f46/f466571d...,4.05,2011-05-16,"['Singleplayer', 'Steam Achievements', 'Multip...","Singleplayer, Steam Achievements, Multiplayer,..."
2,XCOM: Enemy Unknown,10.222222,https://media.rawg.io/media/games/238/2383a172...,4.20,2012-10-09,"['Singleplayer', 'Steam Achievements', 'Multip...","Singleplayer, Steam Achievements, Multiplayer,..."
3,Middle-earth: Shadow of War,10.222222,https://media.rawg.io/media/games/21a/21ad672c...,3.85,2017-09-27,"['Singleplayer', 'Steam Achievements', 'Full c...","Singleplayer, Steam Achievements, Full control..."
4,Batman: Arkham Origins,9.500000,https://media.rawg.io/media/games/fc3/fc30790a...,3.94,2013-10-25,"['Singleplayer', 'Steam Achievements', 'Multip...","Singleplayer, Steam Achievements, Multiplayer,..."
5,Stardew Valley,8.666667,https://media.rawg.io/media/games/713/71326960...,4.41,2016-02-25,"['Singleplayer', 'Steam Achievements', 'Multip...","Singleplayer, Steam Achievements, Multiplayer,..."
6,The Walking Dead: Season 2,7.888889,https://media.rawg.io/media/games/471/4712c9ac...,4.04,2013-12-16,"['Singleplayer', 'Full controller support', 'A...","Singleplayer, Full controller support, Atmosph..."
7,The Wolf Among Us,7.055556,https://media.rawg.io/media/games/be0/be084b85...,4.38,2013-10-10,"['Singleplayer', 'Full controller support', 'A...","Singleplayer, Full controller support, Atmosph..."
8,Game of Thrones - A Telltale Games Series,6.333333,https://media.rawg.io/media/games/c81/c812e158...,3.21,2014-12-02,"['Singleplayer', 'Steam Achievements', 'Full c...","Singleplayer, Steam Achievements, Full control..."
9,The Walking Dead: Season 1,6.333333,https://media.rawg.io/media/games/8d6/8d69eb6c...,4.33,2012-04-23,"['Singleplayer', 'Full controller support', 'A...","Singleplayer, Full controller support, Atmosph..."
